In [1]:
from sklearn import preprocessing
from collections import Counter
import threading
import pickle
import pandas
import numpy
import os

In [2]:
FOLDER_PATH = r'D:\Ready_Corpus\corpusForTrain'

In [3]:
# чтение файлов
def conllu_reader(file: str) -> list:
    with open(file, 'r', encoding='utf-8') as in_conllu:
        sentences = []
        sentence = []
        for line in in_conllu:
            if line != '\n':
                sentence.append(line.split('\t'))
            else:
                sentences.append(sentence)
                sentence = []
    return sentences

In [6]:
# подготовка корпуса
def prepare_corpus(file):   
           
    # columns=['ID', 'word', 'lemma', 'pos', 'smth', 'spec', 'head', 'type']
    # search_word - слово для которого мы ищем по корпусу верщины и зависимые
    # word - слово, которое мы нашли в предложении

    # поиск вершин и зависимых для найденного слова
    def potent_searcher(sentence, search_word, position):
        # 6 - head
        if search_word[6]:
            if position == 'head':
                file_writer(search_word, sentence[int(search_word[6]) - 1], 'word_head')

        if position == 'dependent':
            for potent_dependent in sentence:
                if potent_dependent[6] == search_word[0]:
                    file_writer(search_word, potent_dependent, 'word_dependent')

    # подгонка под формат категорий слова
    def spec_prepare_for_file(spec):
        return '_'.join(spec.split('|'))

    # запись слова в файл
    def file_writer(search_word, potent_word, position_type):
        try:
            file_path = os.path.join(FOLDER_PATH, position_type, f'{"^".join([search_word[1].lower()] + search_word[2:5] + [spec_prepare_for_file(search_word[5])] + [search_word[7].strip()])}.txt')
            with open(file_path, 'a', encoding='utf-8') as out_file:
                out_file.write('\t'.join(potent_word))
        except:
            print(search_word)

    corpus = conllu_reader(file) 
    for sentence in corpus:
        for word in sentence:
            threading.Thread(target=potent_searcher, args=(sentence, word, 'head')).start()
            threading.Thread(target=potent_searcher, args=(sentence, word, 'dependent')).start()               

In [7]:
%%time
prepare_corpus('syntagrus_train.conll')

['10', 'http://www.armeninews.com', 'HTTP://WWW.ARMENINEWS.COM', 'NID', '_', '_', '9', 'аппоз\n']
['5', 'SAP R/3', 'SAP R/3', 'NID', '_', '_', '2', '1-компл\n']
['5', '1/5', '1/5', 'NUM', '_', '_', '6', 'количест\n']
['33', 'в/ч', 'В/Ч', 'S', '_', 'ЖЕН|ИМ|ЕД|НЕОД', '32', 'примыкат\n']['33', 'в/ч', 'В/Ч', 'S', '_', 'ЖЕН|ИМ|ЕД|НЕОД', '32', 'примыкат\n']

['73', 'в/ч', 'В/Ч', 'S', '_', 'ЖЕН|ИМ|ЕД|НЕОД', '72', 'примыкат\n']
['73', 'в/ч', 'В/Ч', 'S', '_', 'ЖЕН|ИМ|ЕД|НЕОД', '72', 'примыкат\n']
['32', 'кВт/ч', 'КВТ/Ч', 'S', '_', 'РОД|МУЖ|МН|НЕОД', '31', 'квазиагент\n']
['22', 'м/с', 'М/С', 'S', '_', 'РОД|МУЖ|МН|НЕОД', '20', 'предл\n']['22', 'м/с', 'М/С', 'S', '_', 'РОД|МУЖ|МН|НЕОД', '20', 'предл\n']

['10', 'м/с', 'М/С', 'S', '_', 'РОД|МУЖ|ЕД|НЕОД', '6', 'компл-аппоз\n']
['10', 'м/с', 'М/С', 'S', '_', 'РОД|МУЖ|ЕД|НЕОД', '6', 'компл-аппоз\n']
['10', 'м/с', 'М/С', 'S', '_', 'РОД|МУЖ|ЕД|НЕОД', '6', 'компл-аппоз\n']
['14', 'м/с', 'М/С', 'S', '_', 'РОД|МУЖ|МН|НЕОД', '12', 'предл\n']
['14', 'м/с', 

In [5]:
def specFileWriter(in_path, out_path):
    for file in os.listdir(os.path.join(FOLDER_PATH, in_path)):
        try:
            with open(os.path.join(FOLDER_PATH, out_path, '^'.join(file.split('^')[2:])), 'a', encoding='utf-8') as out_file:
                with open(os.path.join(FOLDER_PATH, in_path, file), 'r', encoding='utf-8') as in_file:
                    try:
                        for line in in_file:
                            out_file.write(clean_for_spec(line))
                    except:
                        pass
        except:
            print(file)
                
# что-то еще чистим
def clean_for_spec(line):
    cleaned = list(map(lambda x: x.strip(), line.split('\t')))
    return '\t'.join([cleaned[3], cleaned[4], cleaned[5], cleaned[7]]) + '\n' 

In [9]:
%%time
specFileWriter('word_head', 'spec_head')
specFileWriter('word_dependent', 'spec_dependent')

about
CPU times: total: 4min 29s
Wall time: 39min 56s


In [6]:
def root_perMillion_counter():
    root_amount = {}
    for folder in os.listdir(FOLDER_PATH):
        if not folder.endswith('pkl'):
            for file in os.listdir(os.path.join(FOLDER_PATH, folder)):
                if file.endswith('root.txt'):
                    with open(os.path.join(FOLDER_PATH, folder, file), 'rb') as root_file:
                        root_counter = len(root_file.readlines())
                    files_to_check = [file_name for file_name in os.listdir(os.path.join(FOLDER_PATH, folder)) if file_name.split('^')[:-1] == file.split('^')[:-1] and file_name.split('^')[-1] != 'root.txt']
                    other_counter = 0
                    for file_to_check in files_to_check:
                        with open(os.path.join(FOLDER_PATH, folder, file_to_check), 'rb') as other_file:
                            other_counter += len(other_file.readlines())
                    other_counter = other_counter if other_counter else 1
                    root_amount[file[:-9]] = root_counter / 1000000 / other_counter
    return root_amount

In [11]:
%%time
root_posibility = root_perMillion_counter()

CPU times: total: 3h 25min 37s
Wall time: 3h 41min 6s


In [35]:
with open(os.path.join(FOLDER_PATH, 'root_posibility.pkl'), 'rb') as in_file:
    root_posibility = pickle.load(in_file)

In [36]:
def spec_splitter(spec):
    return spec.split('|')

def preapare_file_name_spec(file_name):
    splitted = file_name.split('^')
    return file_name.split('^')[:-2] + ['|'.join(splitted[-2].split('_')) if splitted[-2] != '_' else '_']

def preapare_file_name(file_name):
    splitted = file_name.split('^')
    return splitted[:3] + ['_', '|'.join(splitted[4].split('_')) if splitted[4] != '_' else '_']

In [37]:
# подсчет близости слова и слова с потенциальной связью
def nearest(current_word: list, position: list) -> float:
    position = list(map(lambda x: x.strip(), position.split('\t')))[:-2]
    current_word = current_word[:-1]
    current_word.extend(spec_splitter(current_word[-1]))
    if len(position) == 0: return 0
    position = position[:-1]
    if len(position) == 0: return 0
    position.extend(spec_splitter(position[-1]))
    return len(set(current_word) & set(position)) / ((len(current_word) + len(position)) / 2)    

In [38]:
# считаем ранги. word - word, top_full - variants_counter[0], variants - variants_counter[1]
def is_position(word: list, top_full: list, variants: dict) -> list:
    result = []
    for top in top_full:
        for spec, counter in variants[top[0]].items():
            result.append([spec, nearest(word, spec) * counter])
            # result.append([spec, nearest(word, spec) / sum(variant[1] for variant in top_full)])
    return sorted(result, key=lambda x: x[1], reverse=True)[:5]

In [39]:
# по словарю вершин/зависимых слова возвращается сортированный словарь для получения самого вероятного отношеия и весь словарь(хз зачем если честно). variants - head_dep_searcher
def variants_counter(variants: list) -> 'dict, list':
    variants_counter_dict = {}
    for file in variants:
        result = []
        with open(file, 'rb') as in_file:
            for line in in_file:
                try: result.append(line.decode('utf-8'))
                except: pass
            variants_counter_dict[file.split('\\')[-1]] = Counter(result)
    temp_result = {key: sum(value.values()) for key, value in variants_counter_dict.items()}
    return sorted(temp_result.items(), key=lambda x: x[1], reverse=True), variants_counter_dict

In [40]:
# по словарю отношений между двумя словами готовится отношение между двумя словами. word1, word2 - word, dep_head_dict - head_dep_searcher
def head_dep_marker(dep_head_dict: dict, word1: list, word2: list) -> list:
    max_key = max(dep_head_dict, key=lambda x: dep_head_dict[x][0][1] if len(dep_head_dict[x]) else 0)
    if max_key in ['word2_head_for_word1', 'word1_dep_for_word2']:
        connection = connection_type(word2, word1)
        return [[word1, word2[0], connection[0][0] if len(connection) else '???'], [word2, 'free'], dep_head_dict[max_key][0][1] if len(dep_head_dict[max_key]) else 0]
    else:
        connection = connection_type(word1, word2)
        return [[word1, 'free'], [word2, word1[0], connection[0][0] if len(connection) else '???'], dep_head_dict[max_key][0][1] if len(dep_head_dict[max_key]) else 0]

In [41]:
# определяется тип отношений между вершиной и зависмым. head - word, dep - word
def connection_type(head: list, dep: list) -> list:
    result = []
    for file_name in os.listdir(os.path.join(FOLDER_PATH, 'word_dependent')):
        if preapare_file_name_spec(file_name) == head[1:]:
            with open(os.path.join(FOLDER_PATH, 'word_dependent', file_name), 'r', encoding='utf-8') as words_dep_file:
                for word in words_dep_file:
                    if dep[1:] == word.split('\t')[1:-2]:
                        result.append(word.split('\t')[-1].strip())
    if not len(result):
        for file_name in os.listdir(os.path.join(FOLDER_PATH, 'spec_dependent')):
            if preapare_file_name_spec(file_name) == head[3:]:
                with open(os.path.join(FOLDER_PATH, 'spec_dependent', file_name), 'r', encoding='utf-8') as spec_dep_file:
                    for word in spec_dep_file:
                        if dep[3:] == word.split('\t')[:-1]:
                            result.append(word.split('\t')[-1].strip())
    return sorted(list(Counter(result).items()), key=lambda x: x[1], reverse=True)

In [42]:
# создается список файлов для вершин и зависимых для слова word - word[1:]
def head_dep_searcher(word):
    
    word_paths = ['word_head', 'word_dependent']
    spec_paths = ['spec_head', 'spec_dependent']

    head_variants = []
    dep_variants = []

    for word_path in word_paths:
        for file_name in os.listdir(os.path.join(FOLDER_PATH, word_path)):
            if preapare_file_name(file_name) == word:
                file_path = os.path.join(FOLDER_PATH, word_path, file_name)
                head_variants.append(file_path) if word_path == 'word_head' else dep_variants.append(file_path)


    for spec_path in spec_paths:
        for file_name in os.listdir(os.path.join(FOLDER_PATH, spec_path)):
            if preapare_file_name_spec(file_name) == word[2:]:
                file_path = os.path.join(FOLDER_PATH, spec_path, file_name)
                head_variants.append(file_path) if spec_path == 'spec_head' else dep_variants.append(file_path)

    return head_variants, dep_variants

In [43]:
# словарь рангов, word1, word2 - word[1:]
def head_dep_rank_counter(word1: list, word2: list) -> dict:
    
    head_variants_1, dep_variants_1 = head_dep_searcher(word1)
    head_variants_2, dep_variants_2 = head_dep_searcher(word2)
    
    top_heads_1, head_variants_counter_1 = variants_counter(head_variants_1)
    top_deps_1, dep_variants_counter_1 = variants_counter(dep_variants_1)
    top_heads_2, head_variants_counter_2 = variants_counter(head_variants_2)
    top_deps_2, dep_variants_counter_2 = variants_counter(dep_variants_2)
    
    return {
        'word2_head_for_word1': is_position(word2, top_heads_1, head_variants_counter_1),
        'word2_dep_for_word1': is_position(word2, top_deps_1, dep_variants_counter_1),
        'word1_head_for_word2': is_position(word1, top_heads_2, head_variants_counter_2),
        'word1_dep_for_word2': is_position(word1, top_deps_2, dep_variants_counter_2)
             }

In [44]:
# считается вероятность слова быть root, word
def is_root_counter(root_posibility: dict, word: list) -> float:
    try: return root_posibility['^'.join(word[1:5]) + '^' + '_'.join(word[5].split('|'))]
    except: pass
    try: return root_posibility['^'.join(word[3:5]) + '^' + '_'.join(word[5].split('|'))]
    except: pass
    return  0

In [45]:
# True если нранг новой вершины больше, чем имеющейся
def time_to_change(connection: list, rank: float) -> bool:
    if type(connection[0]) == tuple:
        return connection[0][2] < rank
    return connection[1][2] < rank

# Проверка на отсутствие вершины у слова
def is_free(connection: list) -> bool:
    result = True
    if type(connection[0]) == tuple or type(connection[1]) == tuple:
        result = False
    return result

In [46]:
# поиск виртуально вершины - на вход words[i][1:]. На выход - virt_N, где N - часть речи
def virt_searcher(word: list) -> str:
    head, dependent = head_dep_searcher(word)
    variants = []
    for file in head:
        with open(file, 'r', encoding='utf-8') as in_file:
            variants.extend([word.strip().split('\t')[3] if len(word.strip().split('\t')) == 8 else word.strip().split('\t')[0] for word in in_file.readlines()])
    return 'virt_' + Counter(variants).most_common(1)[0][0] if len(variants) else 'free'

In [47]:
# построение дерева - scentence - предложение (сырое, без деления), root_posibility - словарь вероятностей быть root, tree_temperature - порог постановления виртуальной вершины
def tree_creator(sentence: str, root_posibility: dict, tree_temperature = 0.5) -> dict:
    
    words = [line.split('\t')[:-2] for line in sentence.split('\n')]
    
    # собираем все варианты отношений и считаем их ранги
    all_variants_dict = {}
    for i in range(len(words)):
        all_variants_dict[words[i][1]] = []
        for j in range(i):
            all_variants_dict[words[i][1]].append(head_dep_marker(head_dep_rank_counter(words[i][1:], words[j][1:]), words[i], words[j]))

    # нормируем для того, чтоб потом постанавливать виртуальные вершины
    res = []
    counter = 0
    for key, value in all_variants_dict.items():
        for variant in value:
            res.append(variant[-1])
    norm_res = preprocessing.normalize([numpy.array(res)])
    for key, value in all_variants_dict.items():
        for variant in value:
            variant[-1] = norm_res[0][counter]
            counter += 1

    # делаем страшные махинации чтоб расставлять правильные отношения
    result = {}
    result['\t'.join(words[0])] = (is_root_counter(root_posibility, words[0]), virt_searcher(words[0][1:]))
    max_root = result['\t'.join(words[0])][0]
    for key, values in list(all_variants_dict.items())[1:]:
        sorted_result = sorted(values, key=lambda x: x[-1], reverse=True)
        for variant in sorted_result:
            for elem in variant[:-1]:
                is_root = is_root_counter(root_posibility, elem[0])
                key = '\t'.join(elem[0])
                virt = virt_searcher(elem[0][1:])
                
                # новое слово, котрого еще не встроено в дерево
                if key not in result: result[key] = (is_root, (elem[1], elem[2], variant[-1])) if elem[1] != 'free' and variant[-1] > tree_temperature else (is_root, virt)
                
                # у слова стоит маркер free или virt_HEAD
                elif is_free(result[key]) and elem[1] != 'free' and variant[-1] > tree_temperature: result[key] = (is_root, (elem[1], elem[2], variant[-1])) if type(result[key][0]) == int or type(result[key][0]) == float else ((elem[1], elem[2], variant[-1]), is_root)
                
                # у слова уже найдена вершина и найдена новая вершина с более выоским рангом
                elif not is_free(result[key]) and elem[1] != 'free' and variant[-1] > tree_temperature and time_to_change(result[key], variant[-1]): result[key] = (is_root, (elem[1], elem[2], variant[-1])) if type(result[key][0]) == int or type(result[key][0]) == float else ((elem[1], elem[2], variant[-1]), is_root)
                    
            # меняем местами вероятность быть рутом и ранги с отношениями, чтоб максимальная вероятность была справа
            for key, value in result.items():
                if type(value[0]) == float or type(value[0]) == int:
                    if value[0] > max_root:
                        result[key] = (value[1], value[0])
                        max_root = value[0]
                        for little_key, little_value in result.items():
                            if little_key != key and type(value[1]) == float or type(value[1]) == int:
                                result[little_key] = (value[1], value[0])
                elif type(value[1]) == float or type(value[1]) == int:
                    if value[1] < max_root:
                        result[key] = (value[1], value[0])

    return result

In [48]:
# для витуальных вершин находим самый вероятный тип отншений - word[1:]
def most_popular_connect(word: list) -> str:
    result = ''
    max_result = 0
    for file_name in os.listdir(os.path.join(FOLDER_PATH, 'word_head')):
        if preapare_file_name(file_name) == word:
            with open(os.path.join(FOLDER_PATH, 'word_head', file_name), 'rb') as word_head_file:
                size = len(word_head_file.readlines())
                if size > max_result:
                    result = file_name.split('^')[-1][:-4]
    if not result:
        for file_name in os.listdir(os.path.join(FOLDER_PATH, 'spec_head')):
            if preapare_file_name_spec(file_name) == word[2:]:
                with open(os.path.join(FOLDER_PATH, 'spec_head', file_name), 'rb') as spec_head_file:
                    size = len(spec_head_file.readlines())
                    if size > max_result:
                        result = file_name.split('^')[-1][:-4]
    return result

In [49]:
# преобразавное raw-вида дереа в человекопонятный формат. tree - результат выполнения tree_creator
def tree_shower(tree: dict, isPrinter = False) -> None:
    adder_counter = 1
    adder = []
    ready_sent = []
    for word, head in tree.items():
        word_line = []
        word_line.append(word)
        if type(head[0]) == float or type(head[0]) == int:
            if type(head[1]) == tuple:
                word_line.extend(list(map(str, head[1])))
            else:
                adder.append([str(len(tree.keys()) + adder_counter), head[1], head[1], head[1].split('_')[1], '_', 'used'])
                word_line.extend([str(len(tree.keys()) + adder_counter), most_popular_connect(word.split('\t')[1:])])  
                adder_counter += 1
        else:
             word_line.extend(['0', 'root'])
        ready_sent.append(word_line)
    ready_sent.extend(adder)
    
    if isPrinter:
        print('\n'.join('\t'.join(word) for word in ready_sent))
    else:
        with open(os.path.join('predicted', 'syntagrus_test.conll.predicted'), 'a', encoding='utf-8') as out_file:
            out_file.write('\n'.join('\t'.join(word) for word in ready_sent))
            out_file.write('\n\n')

In [16]:
test_sentences_pred = conllu_reader('predicted/syntagrus_test.conll.predicted')

In [52]:
test_sentences[0]

[['1', 'А', 'А', 'CONJ', '_', '_', '0', 'root\n'],
 ['2', 'ОН', 'ОН', 'S', '_', 'МУЖ|ИМ|ОД|ЕД', '4', 'предик\n'],
 ['3', 'МЯТЕЖНЫЙ', 'МЯТЕЖНЫЙ', 'A', '_', 'МУЖ|ИМ|ЕД', '2', 'оп-опред\n'],
 ['4',
  'ПРОСИТ',
  'ПРОСИТЬ',
  'V',
  '_',
  'ЕД|НЕСОВ|НЕПРОШ|ИЗЪЯВ|3-Л',
  '1',
  'соч-союзн\n'],
 ['5', 'БУРИ', 'БУРЯ', 'S', '_', 'ЖЕН|РОД|ЕД|НЕОД', '4', '1-компл\n']]

In [53]:
test = '''1	В общей сложности	В ОБЩЕЙ СЛОЖНОСТИ	ADV	_	_	3	обст
2	экспедиция	ЭКСПЕДИЦИЯ	S	_	ЖЕН|ИМ|ЕД|НЕОД	3	предик
3	длилась	ДЛИТЬСЯ	V	_	ЕД|НЕСОВ|ЖЕН|ПРОШ|ИЗЪЯВ	0	root
4	более	БОЛЕЕ	ADV	_	_	3	1-компл
5	восьми	ВОСЕМЬ	NUM	_	РОД	6	количест
6	лет	ГОД	S	_	РОД|МУЖ|МН|НЕОД	4	сравнит'''

In [56]:
words = [line.split('\t') for line in test.split('\n')]

In [58]:
for i in range(2, len(words) + 1):
    tree_shower(tree_creator('\n'.join('\t'.join(word).strip() for word in words[:i]), root_posibility, 0.1), True)
    print('\n')

1	В общей сложности	В ОБЩЕЙ СЛОЖНОСТИ	ADV	_	_	3	эксплет
2	экспедиция	ЭКСПЕДИЦИЯ	S	_	ЖЕН|ИМ|ЕД|НЕОД	0	root
3	virt_V	virt_V	V	_	used


1	В общей сложности	В ОБЩЕЙ СЛОЖНОСТИ	ADV	_	_	3	обст	0.6036069242384022
2	экспедиция	ЭКСПЕДИЦИЯ	S	_	ЖЕН|ИМ|ЕД|НЕОД	3	предик	0.793897784144311
3	длилась	ДЛИТЬСЯ	V	_	ЕД|НЕСОВ|ЖЕН|ПРОШ|ИЗЪЯВ	0	root


1	В общей сложности	В ОБЩЕЙ СЛОЖНОСТИ	ADV	_	_	3	обст	0.48281556879062293
2	экспедиция	ЭКСПЕДИЦИЯ	S	_	ЖЕН|ИМ|ЕД|НЕОД	3	предик	0.6350261980458313
3	длилась	ДЛИТЬСЯ	V	_	ЕД|НЕСОВ|ЖЕН|ПРОШ|ИЗЪЯВ	0	root
4	более	БОЛЕЕ	ADV	_	_	3	обст	0.48281556879062293


1	В общей сложности	В ОБЩЕЙ СЛОЖНОСТИ	ADV	_	_	3	обст	0.48160227942886447
2	экспедиция	ЭКСПЕДИЦИЯ	S	_	ЖЕН|ИМ|ЕД|НЕОД	3	предик	0.6334304116206817
3	длилась	ДЛИТЬСЯ	V	_	ЕД|НЕСОВ|ЖЕН|ПРОШ|ИЗЪЯВ	0	root
4	более	БОЛЕЕ	ADV	_	_	3	обст	0.48160227942886447
5	восьми	ВОСЕМЬ	NUM	_	РОД	6	предл
6	virt_S	virt_S	S	_	used


1	В общей сложности	В ОБЩЕЙ СЛОЖНОСТИ	ADV	_	_	3	обст	0.42380126656215983
2	экспедиция	ЭКСПЕДИЦИЯ	S	_	ЖЕН|ИМ|ЕД|НЕОД

In [17]:
test_sentences = conllu_reader('syntagrus_test.conll')

In [32]:
def evaluator_freq(gold, test):
    
    true_head = 0
    true_head_type = 0
    words = 0
    
    short_true_head = 0
    short_true_head_type = 0
    short_words = 0
        
    for test_sentence in test:
        for gold_sentence in gold:
            if list(map(lambda x: x[0:6], test_sentence))[:len(gold_sentence)] == list(map(lambda x: x[0:6], gold_sentence)):
                for i in range(len(gold_sentence)):
                    if test_sentence[i][6] == gold_sentence[i][6]:
                        if len(gold_sentence) < 10: short_true_head += 1
                        true_head += 1
                        if test_sentence[i][7] == gold_sentence[i][7]:
                            if len(gold_sentence) < 10: short_true_head_type += 1
                            true_head_type += 1
                    words += 1
                    if len(gold_sentence) < 10: short_words += 1
    
    return true_head / words, true_head_type / words, short_true_head / short_words, short_true_head_type / short_words
                

In [20]:
def predictor(start, end, sentences):
    for i in range(start, end):
        print(i, end=' ')
        try: tree_shower(tree_creator('\n'.join('\t'.join(word).strip() for word in sentences[i]), root_posibility, 0.1))
        except: pass

In [ ]:
predictor(0, len(test_sentences), test_sentences)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 

In [33]:
evaluator_freq(test_sentences, test_sentences_pred)

(0.2008230452674897,
 0.04691358024691358,
 0.3589108910891089,
 0.0891089108910891)

In [ ]:
# генерация словаря количества зависимых с определенными отшениями
def connect_type_amounter():
    sentences = conllu_reader(r'D:/Ready_Corpus/syntagrus_train.conll')
    dep_amount = {}
    for sentence in sentences:
        for word in sentence:
            dep = [dep[1:] for dep in sentence if dep[6] == word[0]]
            key_word = '^'.join(word[1:-2])
            if key_word not in dep_amount: dep_amount[key_word] = {}
            if len(dep):
                if len(dep) not in dep_amount[key_word]: dep_amount[key_word][len(dep)] = {}
                little_key = ' - '.join(sorted([depe[-1].strip() for depe in dep]))
                if little_key not in dep_amount[key_word][len(dep)]: dep_amount[key_word][len(dep)][little_key] = 1
                else: dep_amount[key_word][len(dep)][little_key] += 1
            else:
                if len(dep) not in dep_amount[key_word]: dep_amount[key_word][len(dep)] = {'none' : 0}
                dep_amount[key_word][len(dep)]['none'] += 1

            key_spec = '^'.join(word[3:-2])
            if key_spec not in dep_amount: dep_amount[key_spec] = {}
            if len(dep):
                if len(dep) not in dep_amount[key_spec]: dep_amount[key_spec][len(dep)] = {}
                little_key = ' - '.join(sorted([depe[-1].strip() for depe in dep]))
                if little_key not in dep_amount[key_spec][len(dep)]: dep_amount[key_spec][len(dep)][little_key] = 1
                else: dep_amount[key_spec][len(dep)][little_key] += 1
            else:
                if len(dep) not in dep_amount[key_spec]: dep_amount[key_spec][len(dep)] = {'none' : 0}
                dep_amount[key_spec][len(dep)]['none'] += 1
                
    result = {}
    for key, value in dep_amount.items():
        if key not in result: result[key] = {}
        for amount, var_list in dep_amount[key].items():
            result[key][amount] = dict(sorted(var_list.items(), key=lambda x: x[1], reverse=True))
    del dep_amount
    for key in result:
        result[key] = dict(sorted(result[key].items(), key=lambda x: sum(list(x[1].values())[:len(x[1].values()) // 2]) if len(x[1].values()) > 1 else list(x[1].values())[0], reverse=True))
    return result